<a href="https://colab.research.google.com/github/d3rp3tt3/AI-Trivia-ChatBot-Project-3/blob/jenn-embeddings/jenn_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Jenn Allen's working file for the AI Chatbook - work in progress

In [1]:
# Install dependencies
!pip install transformers
!pip install accelerate
!pip install sentence-transformers
!pip install chromadb
!pip install langchain
!pip install langchain-community
!pip install pypdf
# Test

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.8 MB/s eta 0:00:00

In [2]:
# Import modules
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [3]:
# Load Phi 3 model
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", torch_dtype='auto', trust_remote_code=True)

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-3-beebfb740d9b>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
<ipython-input-3-beebfb740d9b>:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, cre

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<ipython-input-3-beebfb740d9b>:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [19]:
# Load data

import csv

def load_trivia_questions(filename):
    trivia_questions = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            category = row['Category']
            question = row['Question']
            answer = row['Answer']
            if category not in trivia_questions:
                trivia_questions[category] = []
            trivia_questions[category].append({"question": question, "answer": answer})
    return trivia_questions

trivia_questions = load_trivia_questions('test_data/test_question_set.csv')

In [15]:
print(trivia_questions)

{'HISTORY': [{'question': "For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory", 'answer': 'Copernicus'}, {'question': "Built in 312 B.C. to link Rome & the South of Italy, it's still in use today", 'answer': 'the Appian Way'}, {'question': 'In 1000 Rajaraja I of the Cholas battled to take this Indian Ocean island now known for its tea', 'answer': 'Ceylon (or Sri Lanka)'}], "ESPN's TOP 10 ALL-TIME ATHLETES": [{'question': 'No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves', 'answer': 'Jim Thorpe'}, {'question': 'No. 8: 30 steals for the Birmingham Barons; 2,306 steals for the Bulls', 'answer': 'Michael Jordan'}, {'question': 'No. 1: Lettered in hoops, football & lacrosse at Syracuse & if you think he couldn\'t act, ask his 11 "unclean" buddies', 'answer': 'Jim Brown'}], 'EVERYBODY TALKS ABOUT IT...': [{'question': 'The city of Yuma in this state has a record average of 4,055 hours of 

In [29]:
# Create embeddings for the trivia questions, for use in RAG

from langchain.schema import Document

def create_embeddings(questions):
    documents = []
    for category, question_list in questions.items():
        for question in question_list:
            question_text = question["question"]
            doc = Document(page_content=question_text, metadata={"category": category})
            documents.append(doc)

    return documents

In [30]:
# Create documents
documents = create_embeddings(trivia_questions)

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
vector_db = Chroma.from_documents(documents, embedding=embeddings)
vector_db.persist()


retriever = vector_db.as_retriever(search_kwargs={"k": 1})

<ipython-input-30-89f112c6be77>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
<ipython-input-30-89f112c6be77>:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [ ]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent = initialize_agent(
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    tools=[],
    prompt="You are a trivia bot. Choose a category and select a question from that category.",
    handle_parsing_errors=True,
    early_stopping_method='generate'
)


In [31]:
# Create custom prompt template for the chatbot
qna_prompt_template = """
<system>
You are a trivia bot. When asked a question, choose a category from the list provided and select a trivia question from your knowledge base in that category. Provide the answer to the trivia question.

Available categories: {categories}

User query: Ask me a trivia question from the category you choose.

Your trivia question database:
{question_database}
Y4:0"""


In [34]:
# Function to select a question
def select_question(retrieved_docs, categories):
    if not retrieved_docs:
        return "Sorry, I couldn't find a suitable question."

    doc = retrieved_docs[0]
    category = doc.metadata["category"]
    question = doc.page_content

    return f"Here's a question from the '{category}' category:\n\n{question}"


In [36]:
# Get user input for category selection
user_category = input("Please choose a category: ")

# Generate a prompt for the LLM
prompt = qna_prompt_template.format(categories=list(trivia_questions.keys()), question_database="")

# Run the LLM to get a category suggestion
llm_response = llm.generate([prompt])  # Pass a list containing the prompt
suggested_category = llm_response[0].text.strip()  # Access the first (and only) response

# Retrieve relevant documents
retrieved_docs = retriever.get_relevant_documents(suggested_category)

# Select a question
selected_question = select_question(retrieved_docs, list(trivia_questions.keys()))

print(selected_question)


Please choose a category: History


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


KeyboardInterrupt: 